## 비정형 데이터 분석

- 정형(structured) 데이터: 표 형태의 데이터
- 비정형(unstructured) 데이터: 텍스트, 이미지, 음성 등

# 웹 스크래핑 1

### Python으로 URL 분석하기

In [ ]:
import urllib.parse

In [ ]:
p = urllib.parse.urlparse('http://www.bobaedream.co.kr/mycar/mycar_list.php?sel_m_gubun=ALL&page=2')

In [ ]:
p.scheme

In [ ]:
p.hostname

In [ ]:
p.path

In [ ]:
p.query

### 요청보내기

In [ ]:
import requests

In [ ]:
# GET 요청을 보내고 응답을 받아와 res 변수에 할당한다.
url = 'http://www.bobaedream.co.kr/mycar/mycar_list.php?sel_m_gubun=ALL&page=2'
res = requests.get(url)

In [ ]:
res

In [ ]:
res.status_code
# 응답의 상태 코드를 확인하면 200번을 얻는다.

In [ ]:
requests.get('http://www.bobaedream.co.kr/aaaaaaa')

## 상태코드

- 2XX: 성공
- 3XX: 다른 주소로 이동
- 4XX: 클라이언트 오류
  - 404: 존재하지 않는 주소
- 5XX: 서버 오류
  - 503: 서버가 다운 등의 문제로 서비스 불가 상태

## HTML

- 웹 페이지의 내용을 표현하는 방법
- 노드라는 단위로 구성
- 하나의 노드는 여는 태그, 태그의 내용, 닫는 태그로 구성
- 예: `<a href="http://www.google.com">구글</a>`
  - 여는 태그: `<a href="http://www.google.com">`
  - 내용: `구글`
  - 닫는 태그: `</a>`

## 개발자 도구

- 웹 브라우저에서 F12 또는 우클릭 후 "검사" 메뉴를 클릭하면 개발자도구로 진입
- HTML의 구조와 통신 내역 등을 확인

## HTML의 주요 태그

- div: 구역(division)
- span: 범위(span)
- ul: 번호 없는 리스트(unordered list)
- ol: 번호 리스트(ordered list)
- li: 리스트 항목(list item)
- a: 링크(anchor)

## 응답 내용에서 특정 태그 찾기

먼저 cssselect 패키지를 설치한다.

In [ ]:
#!pip install cssselect

HTML 해석을 위한 lxml.html을 불러온다.

In [ ]:
import lxml.html

응답의 텍스트(res.text)를 해석한다.

In [ ]:
root = lxml.html.fromstring(res.text)

a 태그를 모두 찾는다.

In [ ]:
root.cssselect('a')

## 속성
HTML 태그는 **속성**(attribute)라는 추가 정보를 포함한다. 대표적인 것은 다음과 같다.

- `id`: 노드의 고유 아이디
- `class`: 노드의 서식 유형
- `href`: `a` 태그에만 사용. 링크된 주소.

## CSS 선택자
- HTML에서 특정 노드를 선택하기 위한 표기법
- `.cssselect` 함수에 사용한다

## 클래스의 선택자
- 특정 class의 태그를 지정할 때는 `태그.클래스`와 같이 `.`으로 표시한다
- 선택자에서 `p.tit`는 HTML에서 `<p class="tit">`

In [ ]:
root.cssselect('em.cr')

## 클래스가 2개일 때
- `태그.클래스1.클래스2`와 같이 `.`으로 구분하여 표시한다
- 선택자에서 `p.tit.ellipsis`는 HTML에서 `<p class="tit ellipsis">`
- `p.tit`이나 `p.ellipsis`만 해도 `p.tit.ellipsis`는 선택 된다

## 포함관계인 노드의 선택자
```html
<p class="tit ellipsis">
    <a href="...">아우디 A8</a>
</p>
```
- 위의 예는 `p` 태그 안에 `a` 태그가 포함됨
- 선택자에서 포함관계는 공백으로 표시: `p.tit a`

In [ ]:
links = root.cssselect('p.tit a')

In [ ]:
links

## href 속성 모으기

링크의 걸린 주소를 수집한다

In [ ]:
link = links[0]

In [ ]:
link.attrib['href'] 

In [ ]:
for link in links:
    print(link.attrib['href'])

In [ ]:
x = [1,2,3]
x.append(5)
x

In [ ]:
urls = []
for link in links:
    urls.append(link.attrib['href'])

In [ ]:
urls

## 상대주소

- `/mycar/mycar_view.php?no=1944109&gubun=K`는 스키마와 호스트가 생략된 상대주소

- 원래 주소 `http://www.bobaedream.co.kr/mycar/mycar_list.php?sel_m_gubun=ALL&page=2`를 이용해 절대주소로 변환

In [ ]:
import urllib.parse

In [ ]:
urllib.parse.urljoin(url, '/mycar/mycar_view.php?no=1944109&gubun=K')

링크된 주소를 절대 주소로 수집

In [ ]:
urls = []
for link in links:
    href = urllib.parse.urljoin(url, link.attrib['href'])
    urls.append(href)

In [ ]:
urls

링크에서 텍스트를 추출

In [ ]:
texts = []
for link in links:
    texts.append(link.text_content())

In [ ]:
texts

수집된 주소를 저장

In [ ]:
import pandas

In [ ]:
df = pandas.DataFrame({'url': urls, 'text': texts})

In [ ]:
df.head()

In [ ]:
df.to_excel('중고차.xlsx')

# 웹 스크랩 2

## 네이버 의사모
- PC버전: https://cafe.naver.com/duoin
- 모바일버전: https://m.cafe.naver.com/duoin

모바일 버전은 주소 m이 들어가있는 형태가 많음

## 본문 긁기
https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=83048&page=1&boardtype=L&menuid=366

In [ ]:
import requests
article_url = 'https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=83048&page=1&boardtype=L&menuid=366'

In [ ]:
res = requests.get(article_url)

## id 선택자

- 본문 영역은 다음 태그로 감싸여 있다

```html
<div id="postContent" class="post_cont font_zoom1" style="overflow-x:auto;">
```

- `id`는 선택자에서 `#`으로 표시

In [ ]:
import lxml.html
root = lxml.html.fromstring(res.text)

In [ ]:
root.cssselect('div#postContent')

## 본문 내용 보기

- `cssselect`는 결과를 항상 리스트 형식으로 반환
- 본문은 리스트의 첫번째 값이므로 0번을 지정

In [ ]:
content = root.cssselect('div#postContent')[0]

In [ ]:
content.text_content()

## 정규표현식으로 공백지우기

- `\n`, `\t`는 엔터와 탭을 나타냄. 
- 공백문자는 택스트 분석에서 무시되므로 굳이 지울 필요는 없음
- 깔끔하게 처리하고 싶으면
- 불필요한 공백을 지우려면 다음과 같이 정규표현식 사용

In [ ]:
import re

In [ ]:
re.sub('[\s\u200b]+', ' ', content.text_content())

## 정규표현식 설명

- 정규표현식에서 `[]`는 교체할 글자 범위
- `\s`는 일반적 공백
- `\u200b는 유니코드의 `200B` 폭없는 공백 문자
- 끝의 +는 1개 이상이라는 뜻
- `[\s\u200b]+` '공백과 폭 없는 공백문자가 1개 이상 반복되는 경우'

## 회원 전용 게시판

- 모바일 버전으로 접속
- 우상단 ☰ 버튼 클릭 후 아래쪽에서 찾음

https://m.cafe.naver.com/ArticleList.nhn?search.clubid=19773565&search.menuid=98&search.boardtype=L

- 대부분 게시물이 회원만 볼 수 있음

https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=82659&page=1&boardtype=L&menuid=98

회원 전용 게시판 스크랩

In [ ]:
article_url = 'https://m.cafe.naver.com/ArticleRead.nhn?clubid=19773565&articleid=82659&page=1&boardtype=L&menuid=98'

In [ ]:
res = requests.get(article_url)

In [ ]:
root = lxml.html.fromstring(res.text)

본문을 읽을 수 없음

In [ ]:
root.cssselect('#postContent')

## 리퍼러 바꾸기

- 대부분 네이버 카페 게시물은 검색을 통해서 들어가면 게시물을 볼 수 있음
- 마치 검색을 거쳐온 것처럼 하면 회원 전용 게시물도 열람 가능
- 리퍼러(referer): 거쳐온 주소

In [ ]:
search_url = 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=1'

In [ ]:
res = requests.get(article_url, headers={'Referer': search_url})

In [ ]:
root = lxml.html.fromstring(res.text)

본문 영역이 선택됨

In [ ]:
root.cssselect('#postContent')

## 한글이 깨지는 경우

대법원 홈페이지에 접속하면 한글이 깨져보인다

In [ ]:
res = requests.get('http://scourt.go.kr/scourt/index.html')

In [ ]:
root = lxml.html.fromstring(res.text)

In [ ]:
root.cssselect('title')[0].text_content()

## 한글 인코딩
- 컴퓨터는 모든 것을 수(number)로 다룸
- 한글 인코딩: 한글을 수로 나타내는 방법
- 현재 국내에서 흔히 사용되는 인코딩은 2가지
  - UTF-8: 유니코드라는 국제 표준의 한 형식
  - EUC-KR: 완성형 국내 표준

## requests의 인코딩 처리
- 인코딩을 자동 인식하나 가끔 부정확
- 대법원 사이트의 경우 EUC-KR을 ISO-8859-1로 오인식
- ISO-8859-1은 서유럽 언어를 위한 유니코드 이전 국제표준(Latin-1)

## 인코딩 바꾸기
- 인코딩 값을 바꿔주면 됨
- 국내 사이트의 경우 `euc-kr`과 `utf8` 둘 중에 하나이므로 하나씩 시도

In [ ]:
res.encoding

In [ ]:
res.encoding = 'euc-kr'
#res.encoding = 'utf-8'

다시 처리해보면

In [ ]:
root = lxml.html.fromstring(res.text)

In [ ]:
root.cssselect('title')[0].text_content()

# 셀레늄

## 학습목표

웹브라우저가 필요한 상황에서 셀레늄을 이용해 스크래핑하는 방법을 알아본다

## 최근의 웹 트렌드

- HTML + 프로그램(JavaScript)
- 웹 페이지의 내용이 동적으로 변화
- 단순히 HTML만 긁어서 내용 파악이 불가능

## 트립어드바이저

https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d1371740-Reviews-Mugyodong_Bugeokukjib-Seoul.html

- 리뷰의 모든 내용은 '더보기' 버튼을 클릭해야 볼 수 있음
- 개발자 도구에서 Network를 확인해보면 더보기 버튼 클릭시 통신이 이뤄지며 리뷰 내용을 가져오는 것을 확인

## 리뷰 수집 방법

1. '더보기' 버튼을 클릭했을 때 통신 내역을 requests로 따라한다
2. 웹 브라우저를 띄워서 '더보기' 버튼을 클릭한다

## 셀레늄

- 웹 브라우저 제어 도구
- 원래는 웹 사이트 자동 테스트를 위한 목적
- 웹 사이트에 requests 등으로 긁기 어려운 부분이 있을 때 활용
- 속도는 느림

## 드라이버

브라우저별로 '드라이버'가 필요

  1. 셀레늄 홈페이지 https://www.seleniumhq.org/
  2. Download 탭 이동
  3. 필요한 드라이버 링크 선택

## 크롬 드라이버

- 인터넷 익스플로러는 셀레늄과 문제가 있음
- 구글 크롬(Chrome) 사용을 추천
  1. 셀레늄 홈페이지에서 Google Chrome Driver를 클릭
  2. Latest Release 링크를 클릭
  3. Chrome Driver X.XX 링크를 클릭
  4. 운영체제(win, mac, linux)에 맞는 파일을 다운로드
  5. 압축을 해제
  6. chromedriver.exe 파일을 현재 작업 폴더로 복사

## 크롬 브라우저 띄우기

In [9]:
from selenium.webdriver import Chrome

In [13]:
browser = Chrome()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


크롬이 뜨면서 **Chrome이 자동화된 테스트 소프트웨어에 의해 제어되고 있습니다**라고 표시

In [3]:
url = 'https://www.tripadvisor.co.kr/Restaurant_Review-g294197-d1371740-Reviews-Mugyodong_Bugeokukjib-Seoul.html'

In [4]:
browser.get(url)
#browser.get('https://www.naver.com')

NameError: name 'browser' is not defined

## 링크 찾기

- `find_elements_by_css_selector`로 '더보기' 링크를 찾는다. 
  - 단수형 `find_element_~`: 첫번째를 찾음
  - 복수형 `find_elements_~`: 모두 찾음(`lxml.html`의 `cssselect`와 동일)

In [5]:
more_links = browser.find_elements_by_css_selector('.ulBlueLinks')

NameError: name 'browser' is not defined

In [ ]:
more_links

## 클릭

`.click` 메소드로 클릭

In [ ]:
for link in more_links:  # 모든 링크를
    try:
        link.click()     # 하나씩 클릭한다
    except:              # 예외가 발생하면
        pass             # 넘어간다

## 리뷰

- `.text`로 텍스트 확인
- `lxml.html`의 `.text_content()`와 동일

In [ ]:
reviews = browser.find_elements_by_css_selector('.partial_entry')

In [ ]:
for review in reviews:
    print(review.text)

In [ ]:
data = []
for review in reviews:
    data.append(review.text)

In [ ]:
import pandas as pd

In [ ]:
#pd.DataFrame({'text':data}).to_excel('북어국.xlsx')
df = pd.DataFrame({'text':data})
df.to_excel('북어국.xlsx')

## 셀레늄 문서

https://selenium-python.readthedocs.io

In [ ]:
df

In [14]:
import os

In [15]:
os.getcwd()

'/Users/HumanRevolution/jupytercreation/class'

In [16]:
import time

In [17]:
print('hell')
time.sleep(3)
print('o')

hell
o


# 워드 클라우드

데이터 가져오기

In [1]:
import pandas as pd
import lxml.html
import requests

In [2]:
res = requests.get('https://www.nytimes.com/2019/06/03/technology/itunes-shutting-down.html')

In [3]:
res

<Response [200]>

In [4]:
root = lxml.html.fromstring(res.text)

In [5]:
content = root.cssselect('[itemProp="articleBody"]')[0].text_content()

In [6]:
content

'Dearly beloved,We are gathered here today to say goodbye to our friend iTunes, who went to the great trash can in the sky on Monday.It was a gentle, dignified passing. ITunes was surrounded by family and friends at Apple’s annual developer conference when Apple executives announced that it would be dismantled, and its features would be split among three apps: Apple Music, Apple Podcasts and Apple TV.ITunes entered this world 18 years ago as a “digital jukebox” that let users import their favorite CDs, organize their libraries and burn custom mixes. It then became a music store of its own — a magical, one-click emporium where 99 cents could get you almost any song under the sun. Steve Jobs heralded its birth as the dawn of a new age of media consumption, one in which consumers would own the digital rights to their own music.“We think people want to buy their music on the internet by buying downloads, just like they bought LPs, just like they bought cassettes, just like they bought CDs,

TDM

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words='english')

In [ ]:
tdm = cv.fit_transform([content]).toarray()

In [ ]:
df = pd.DataFrame({
    'word': cv.get_feature_names(),
    'freq': tdm.flat
})

In [ ]:
df.sort_values('freq', ascending=False).head(20)

단어 구름 그리기

In [ ]:
from wordcloud import WordCloud

In [ ]:
wc = WordCloud(width=400, height=400, background_color='white')
#wc = WordCloud(font_path='C:\Windows\Fonts\malgun.ttf', width=400, height=400, background_color='white')
# 한글 파일일 경우 폰트 지정할 것.

In [ ]:
frequencies = {word: freq for word, freq in zip(cv.get_feature_names(), tdm.flat)}
# dictionery 형태로 변경

In [ ]:
wc.fit_words(frequencies)

In [ ]:
wc.to_image()